## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-09-51-33 +0000,bbc,Texans did not immediately receive flood alert...,https://www.bbc.com/news/articles/cq537dp2ello
1,2025-07-11-09-44-24 +0000,bbc,US cuts visa validity for applicants from four...,https://www.bbc.com/news/articles/cx23ywg9w82o
2,2025-07-11-09-44-11 +0000,nyt,Meet the French Game Show Contestant Who Won 6...,https://www.nytimes.com/2025/07/11/arts/televi...
3,2025-07-11-09-30-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
4,2025-07-11-09-26-43 +0000,bbc,MP Rupert Lowe investigated by standards watchdog,https://www.bbc.com/news/articles/ceq7z421pwzo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,18
57,brazil,7
104,tariffs,6
131,uk,6
159,rare,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
95,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,47
76,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,46
54,2025-07-10-20-22-01 +0000,nyt,US-Brazil Tariffs: What to Know About Trump’s ...,https://www.nytimes.com/2025/07/10/world/ameri...,36
66,2025-07-10-18-19-52 +0000,bbc,Why is Trump targeting Brazil - and will it ba...,https://www.bbc.com/news/articles/crk68drj57mo,36
97,2025-07-10-11-30-07 +0000,bbc,Brazil vows to match US tariffs after Trump th...,https://www.bbc.com/news/articles/c62dvyv60z7o,36


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
95,47,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
72,23,2025-07-10-16-43-05 +0000,bbc,UK faces rising and unpredictable threat from ...,https://www.bbc.com/news/articles/c98wjzj4jlpo
76,22,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
3,18,2025-07-11-09-30-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
24,18,2025-07-11-05-25-20 +0000,nyt,Rare Earths Mining in Myanmar Is Polluting Tha...,https://www.nytimes.com/2025/07/11/world/asia/...
79,15,2025-07-10-14-42-38 +0000,nyt,Trump Seems to Be Warming to What Europe Wants...,https://www.nytimes.com/2025/07/10/world/europ...
67,14,2025-07-10-18-00-31 +0000,bbc,"'One in, one out' sounds simple - but the migr...",https://www.bbc.com/news/articles/c8d60djgqndo
27,13,2025-07-11-05-00-11 +0000,nyt,"Friday Briefing: Iran Has Some Uranium Left, I...",https://www.nytimes.com/2025/07/11/briefing/ir...
61,13,2025-07-10-19-21-49 +0000,nyt,Rubio Tells Top Russian Diplomat of Trump’s Fr...,https://www.nytimes.com/2025/07/09/world/asia/...
58,13,2025-07-10-19-45-28 +0000,bbc,Watch: Rare shelf cloud seen over Maryland ami...,https://www.bbc.com/news/videos/cwyg93lzqd7o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
